In [24]:
import plotly.plotly as py
import plotly.graph_objs as go
import networkx as nx
import pandas as pd
import numpy as np
import xlwings
import pyperclip

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 1000)
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

df_case = pd.read_csv('Case_Data.csv')
df_case = df_case.drop(['Unnamed: 0'], axis=1)
df_agents = pd.read_csv('Agents.csv')
df_lenders = pd.read_csv('Lenders.csv')

In [25]:
df_agents.loc[df_agents['Case'] == 'In re Toys "R" Us, Inc.']

,Case,case_name,case_status,petition_date,confirmation_date,dismissal_date
92,"In re Toys ""R"" Us, Inc.",Agents:,JPMorgan,Admin Agent,25-Oct-17,NaN
93,"In re Toys ""R"" Us, Inc.",Agents:,JPMorgan,Admin Agent,24-Oct-17,NaN
94,"In re Toys ""R"" Us, Inc.",Agents:,NexBank SSB,Admin Agent,24-Oct-17,NaN
95,"In re Toys ""R"" Us, Inc.",Agents:,Wilmington Savings Fund Society,Indenture Trustee,25-Oct-17,NaN
235,"In re Toys ""R"" Us, Inc.",Agents:,Wilmington Savings Fund Society,Indenture Trustee,NaN,NaN


In [2]:
df_agents = df_agents.set_index(['Case', 'case_name'])
df_agents['petition_date'] = df_agents['petition_date'].fillna('None')

In [6]:
dup_trustees = list(df_agents.loc[df_agents['petition_date'] == 'Indenture Trustee'].duplicated(keep=False))
if True in dup_trustees:
    raise ValueError('Data Issue: Duplicate Indenture Trustees')
df_agents['petition_date'] = df_agents['petition_date'].replace(to_replace=['Indenture Trustee', 'None'], value='Admin Agent')
#^^^^ Need to account for future mistakes in this .replace code

In [4]:
non_admin = set(df_agents[~df_agents.petition_date.isin(['Admin Agent'])])
admin = set(df_agents[df_agents.petition_date.isin(['Admin Agent'])])
if not non_admin.issubset(admin):
    raise ValueError('Data Issue: Case with not Admin Agent')

In [7]:
df_admin = df_agents[df_agents.petition_date.isin(['Admin Agent'])]

In [10]:
df_admin = df_admin.reset_index()

In [14]:
df_admin

,Case,case_name,case_status,petition_date,confirmation_date,dismissal_date
0,In re Fairway Group Holdings Corp.,Agents:,Credit Suisse,Admin Agent,NaN,NaN
1,"In re Arch Coal, Inc.",Agents:,Wilmington Trust,Admin Agent,NaN,NaN
2,"In re Aspect Software Parent, Inc.",Agents:,Wilmington Trust,Admin Agent,NaN,NaN
3,In re Fairway Group Holdings Corp.,Agents:,Credit Suisse,Admin Agent,NaN,NaN
4,In re Peabody Energy Corporation,Agents:,CitiBank,Admin Agent,NaN,NaN
5,In re CJ Holding Co. (C&amp;J Energy),Agents:,Cortland Capital Market Services,Admin Agent,NaN,NaN
6,In re American Gilsonite Company,Agents:,Wilmington Trust,Admin Agent,NaN,NaN
7,In re Phoenix Brands LLC,Agents:,Madison Capital Funding,Admin Agent,NaN,NaN
8,In re Peabody Energy Corporation,Agents:,CitiBank,Admin Agent,NaN,NaN
9,In re Peabody Energy Corporation,Agents:,CitiBank,Admin Agent,NaN,NaN


In [12]:
admin_case_count = df_admin.groupby('Case').count()

In [13]:
admin_case_count

,case_name,case_status,petition_date,confirmation_date,dismissal_date
Case,,,,,
"In re 21st Century Oncology Holdings, Inc.",1,1,1,1,0
"In re ABT Molecular Imaging, Inc.",1,1,1,1,0
In re ADPT DFW Holdings LLC (Adeptus Health),1,1,1,1,0
In re Abengoa Bioenergy Biomass of Kansas LLC,1,1,1,1,0
"In re Abengoa Bioenergy US Holding, LLC",1,1,1,1,0
In re AcuSport Corporation,1,1,1,0,0
"In re Aerogroup International, Inc. (Aerosoles)",1,1,1,0,0
"In re Aeropostale, Inc.",2,2,2,0,0
"In re Aerospace Holdings, Inc. (GroupAero)",1,1,1,0,0


In [15]:
admin_case_count_greater_than_1 = admin_case_count[admin_case_count['case_name']>1]

In [16]:
admin_case_count_greater_than_1

,case_name,case_status,petition_date,confirmation_date,dismissal_date
Case,,,,,
"In re Aeropostale, Inc.",2,2,2,0,0
"In re Appvion, Inc.",2,2,2,1,0
"In re BCBG Max Azria Global Holdings, LLC",3,3,3,0,0
In re BPS US Holdings Inc. (Bauer Hockey / Performance Sports Group),2,2,2,2,0
In re Breitburn Energy Partners LP,2,2,2,2,0
"In re Cenveo, Inc.",2,2,2,1,0
In re Charming Charlie Holdings Inc.,2,2,2,2,0
"In re Claire's Stores, Inc.",2,2,2,2,0
"In re Emerald Oil, Inc.",2,2,2,2,1


In [17]:
df_case.head(3)

,case_name,case_status,petition_date,confirmation_date,dismissal_date,closing_date,effective_date,publishing_name,currency,loan_amount,maturity,facility_type,facility_sub_type,facility_other_type,fixed/floating,pik,amount,floating_type,floating_amount,investigation_budget,ucc_sublimit,carve_out,professional_breakdown,deadline,sublimit_amount,priotity/prime_everything,notes,collateral_all_assets,notes.1,adequate_protection,min_liquidity_covenant,variance,professional_line_item,professional_line_item_period
0,In re Fairway Group Holdings Corp.,confirmed,5/2/2016,6/8/2016,NaN,NaN,7/3/2016,DIP Term Loan,USD,55000000.0,3.0,Term Loan,NaN,NaN,Floating,NaN,NaN,LIBOR,8.00,250000.0,Yes,1000000.0,NaN,30.0,30.0,Y,NaN,Y,NaN,"Liens,FeesExpenses",NaN,15.0,542000000,13
1,"In re Arch Coal, Inc.",confirmed,1/11/2016,9/15/2016,NaN,10/5/2016,10/5/2016,DIP Term Loan,USD,275000000.0,12.0,Term Loan,NaN,NaN,Floating,NaN,NaN,LIBOR,9.00,250000.0,NaN,10000000.0,NaN,64.0,NaN,Y,Additional up to $75mm carve-out in favor of c...,N,Collateral package excludes Receivables Rights,"Liens,Interest,FeesExpenses,FinancialInformation",Yes,NaN,NaN,NaN
2,"In re Aspect Software Parent, Inc.",confirmed,3/9/2016,5/25/2016,NaN,NaN,5/25/2016,DIP Term Loan,USD,30000000.0,7.0,Term Loan,NaN,NaN,Floating,NaN,NaN,LIBOR,9.75,50000.0,Yes,2000000.0,NaN,75.0,60.0,Y,NaN,Y,All Assets,"Liens,Interest,FeesExpenses",Yes,20.0,4480000,NaN


In [23]:
df_case.loc[df_case['case_name'] == 'In re Toys "R" Us, Inc.']

,case_name,case_status,petition_date,confirmation_date,dismissal_date,closing_date,effective_date,publishing_name,currency,loan_amount,maturity,facility_type,facility_sub_type,facility_other_type,fixed/floating,pik,amount,floating_type,floating_amount,investigation_budget,ucc_sublimit,carve_out,professional_breakdown,deadline,sublimit_amount,priotity/prime_everything,notes,collateral_all_assets,notes.1,adequate_protection,min_liquidity_covenant,variance,professional_line_item,professional_line_item_period
109,"In re Toys ""R"" Us, Inc.",pending,9/18/2017,NaN,NaN,NaN,NaN,DIP Revolver,USD,1.850000e+09,16.0,Revolver,NaN,NaN,Floating,NaN,NaN,LIBOR,2.50,500000.0,NaN,20000000.0,NaN,120.0,NaN,Y,"DIP liens prime all prepetition liens, but hav...",Y,NaN,"Liens,Interest,FeesExpenses",NaN,17.5,163000000,72
110,"In re Toys ""R"" Us, Inc.",pending,9/18/2017,NaN,NaN,NaN,NaN,DIP USD 450m Term Loan (FILO),USD,4.500000e+08,16.0,Term Loan,NaN,NaN,Floating,NaN,NaN,LIBOR,2.50,50000.0,NaN,20000000.0,NaN,NaN,NaN,Y,"DIP liens prime all prepetition liens, but hav...",Y,NaN,"Liens,Interest,FeesExpenses",NaN,17.5,163000000,72
111,"In re Toys ""R"" Us, Inc.",pending,9/18/2017,NaN,NaN,NaN,NaN,DIP USD 450m Term Loan,USD,4.500000e+08,16.0,Term Loan,NaN,NaN,Floating,NaN,NaN,BASE RATE,8.75,500000.0,NaN,20000000.0,NaN,120.0,NaN,Y,"DIP liens prime all prepetition liens, but hav...",Y,NaN,"Liens,Interest,FeesExpenses",NaN,17.5,163000000,72
112,"In re Toys ""R"" Us, Inc.",pending,9/18/2017,NaN,NaN,NaN,NaN,TAJ DIP Notes,USD,3.750000e+08,16.0,Bond/Note,NaN,NaN,Fixed,NaN,11.0,LIBOR,NaN,500000.0,NaN,20000000.0,NaN,120.0,NaN,N,Junior to any liens on assets of the domestic ...,Y,NaN,"Liens,Interest,FeesExpenses",NaN,15.0,60000000,72
113,"In re Toys ""R"" Us, Inc.",pending,9/18/2017,NaN,NaN,NaN,NaN,DIP Term Loan (Intercompany),USD,7.500000e+07,16.0,Term Loan,NaN,NaN,NaN,NaN,NaN,LIBOR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,"DIP liens prime all prepetition liens, but hav...",Y,NaN,NaN,NaN,NaN,NaN,NaN
318,"In re Toys ""R"" Us, Inc.",pending,9/18/2017,NaN,NaN,NaN,NaN,Additional (add-on) TAJ DIP Notes,USD,8.000000e+07,16.0,Bond/Note,NaN,NaN,Fixed,NaN,11.0,LIBOR,NaN,500000.0,NaN,20000000.0,NaN,120.0,NaN,N,NaN,Y,NaN,"Liens,Interest,FeesExpenses",NaN,15.0,60000000,NaN


In [18]:
case_count = df_case.groupby('case_name').count()

In [19]:
case_count

,case_status,petition_date,confirmation_date,dismissal_date,closing_date,effective_date,publishing_name,currency,loan_amount,maturity,facility_type,facility_sub_type,facility_other_type,fixed/floating,pik,amount,floating_type,floating_amount,investigation_budget,ucc_sublimit,carve_out,professional_breakdown,deadline,sublimit_amount,priotity/prime_everything,notes,collateral_all_assets,notes.1,adequate_protection,min_liquidity_covenant,variance,professional_line_item,professional_line_item_period
case_name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"In re 21st Century Oncology Holdings, Inc.",1,1,1,0,0,1,1,1,1,1,1,0,0,1,0,0,1,1,1,0,1,0,1,0,1,0,1,1,1,0,1,1,1
"In re 4 West Holdings, Inc. (Orianna Health Systems)",2,2,0,0,0,0,2,2,2,2,2,0,0,2,0,0,2,2,2,2,2,2,2,0,2,0,2,0,2,0,2,0,0
"In re ABT Molecular Imaging, Inc.",1,1,0,0,0,0,1,1,1,1,1,0,0,1,0,1,1,0,0,0,1,0,0,0,1,0,1,0,1,0,1,0,0
In re ADPT DFW Holdings LLC (Adeptus Health),1,1,1,0,0,1,1,1,1,1,1,0,0,1,0,1,1,0,0,0,0,0,1,0,1,0,1,0,1,0,1,1,1
"In re AF-Southeast, LLC (Allied Fiber)",1,1,0,1,1,0,1,1,1,1,1,0,0,1,0,1,1,0,1,1,1,0,1,1,0,0,1,0,1,1,1,1,0
"In re AIX Energy, Inc.",1,1,0,0,0,0,1,1,1,1,1,0,1,1,0,0,1,1,1,0,0,0,0,0,1,1,1,0,1,0,0,0,0
"In re AOG Entertainment, Inc. (Core Entertainment)",1,1,1,0,0,1,1,1,1,1,1,0,0,1,1,1,1,0,1,0,1,0,1,0,1,0,1,1,1,0,1,1,0
In re Abengoa Bioenergy Biomass of Kansas LLC,1,1,1,0,0,1,1,1,1,1,1,0,0,1,0,1,1,0,0,0,0,0,0,0,1,0,1,1,1,0,1,1,1
"In re Abengoa Bioenergy US Holding, LLC",2,2,2,0,0,2,2,2,2,1,2,0,0,1,0,1,2,0,0,0,1,0,0,0,1,1,1,0,1,0,1,1,1


In [21]:
case_count_greater_than_1 = case_count[case_count['loan_amount']>1]

In [22]:
case_count_greater_than_1

,case_status,petition_date,confirmation_date,dismissal_date,closing_date,effective_date,publishing_name,currency,loan_amount,maturity,facility_type,facility_sub_type,facility_other_type,fixed/floating,pik,amount,floating_type,floating_amount,investigation_budget,ucc_sublimit,carve_out,professional_breakdown,deadline,sublimit_amount,priotity/prime_everything,notes,collateral_all_assets,notes.1,adequate_protection,min_liquidity_covenant,variance,professional_line_item,professional_line_item_period
case_name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"In re 4 West Holdings, Inc. (Orianna Health Systems)",2,2,0,0,0,0,2,2,2,2,2,0,0,2,0,0,2,2,2,2,2,2,2,0,2,0,2,0,2,0,2,0,0
"In re Abengoa Bioenergy US Holding, LLC",2,2,2,0,0,2,2,2,2,1,2,0,0,1,0,1,2,0,0,0,1,0,0,0,1,1,1,0,1,0,1,1,1
"In re Aeropostale, Inc.",2,2,2,0,0,2,2,2,2,2,2,0,0,2,0,0,2,2,2,2,2,0,0,2,2,0,2,0,2,1,2,2,0
"In re Appvion, Inc.",3,3,0,0,0,0,3,3,3,3,3,1,1,3,1,1,3,2,0,0,2,2,1,0,2,1,2,2,2,1,2,1,1
"In re BCBG Max Azria Global Holdings, LLC",3,3,3,0,0,3,3,3,3,3,3,0,0,3,1,0,3,3,3,0,3,0,3,0,3,0,3,1,3,0,3,0,0
In re BPS US Holdings Inc. (Bauer Hockey / Performance Sports Group),2,2,2,0,0,2,2,2,2,2,2,0,0,2,0,1,2,1,2,0,2,0,2,0,2,0,2,0,2,2,0,2,2
In re Breitburn Energy Partners LP,2,2,2,0,0,2,2,2,2,2,2,0,0,2,0,0,2,2,2,0,2,0,2,0,2,2,2,0,2,0,2,0,0
"In re CS Mining, LLC",2,2,2,0,0,2,2,2,2,2,2,0,0,2,0,2,2,0,2,0,2,0,0,0,2,2,2,0,2,1,1,2,2
"In re Cenveo, Inc.",2,2,0,0,0,0,2,2,2,2,2,0,0,2,0,0,2,2,2,0,2,0,2,0,2,0,2,0,2,0,2,2,2
